### Import modules

In [1]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import openai

### Define endpoints

In [2]:
EMBEDDINGS_IP="34.142.174.68"
VECTORDB_IP="34.142.174.68"
LLM_IP="34.142.174.68"

LLM_PORT="8000"
EMBEDDINGS_PORT="8001"
VECTORDB_PORT="8002"

### Initialize ChromaDB

In [3]:
client = chromadb.HttpClient(host=VECTORDB_IP, port=VECTORDB_PORT, settings=Settings(allow_reset=True, anonymized_telemetry=False))
default_ef = embedding_functions.HuggingFaceEmbeddingServer(url="http://"+ EMBEDDINGS_IP +":"+ EMBEDDINGS_PORT +"/embed")
collection = client.get_or_create_collection(name="baggage_pol",embedding_function=default_ef)

### Generate context for a query

In [4]:
def generate_context(query):
    vector = default_ef([query])
    results = collection.query(    
        query_embeddings=vector,
        n_results=2,
        include=["documents"]
    )
    res = " \n".join(str(item) for item in results['documents'][0])
    return res

In [5]:
print(generate_context("baggage policy for domestic"))

The baggage allowance for RedJet ﬂights is as follows: Each passenger can bring one carry-on bag and one personal item, such as a purse, laptop, or backpack. The carry-on bag must not exceed 55 x 40 x 20 cm and 10 kg, and the personal item must ﬁt under the seat in front of you. 
The oversized baggage fee is 45 EUR per bag. Passengers are advised to check the baggage policy and fees on the RedJet website before ﬂying and to pack accordingly. RedJet reserves the right to refuse or charge extra for any baggage that does not comply with the policy.


### Helper function for chat completion

In [6]:
MODEL="mistral-7b-instruct-v0.2.Q4_K_M.gguf"

In [7]:
def chat_completion(system_prompt, user_prompt, length=1000):
    client = openai.OpenAI( base_url="http://"+LLM_IP+":"+LLM_PORT+"/v1",api_key="_")
    completion = client.chat.completions.create(
    model=MODEL,
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return (completion.choices[0].message.content)

### Helper function for Chatbot

In [8]:
system_prompt = """
You are a helpful AI assistant that can answer questions based on the context provided.
\n If you cannot find the correct answer in the context, say I don't know but do not include any other information.
\n Be concise and just include the response.
"""

In [9]:
def chatbot(query):
    context = generate_context(query)
    user_prompt = f"""
        Based on the context:
        {context}
        Answer the below query:
        {query}
    """
    resp = chat_completion(system_prompt, user_prompt)
    return resp

### Invoke Chatbot with RAG

In [16]:
#res=chatbot("What's the excess baggage fee for domestic flights?")
#res=chatbot("What's the excess baggage fee for international flights?")
#res=chatbot("What's the size of carry-on bag?")
res=chatbot("What's maximum weight for checked baggage?")

In [17]:
print(res)

 The maximum weight for checked baggage is 35 kg per bag.
